In [1]:
%load_ext autoreload

In [2]:
%autoreload
from trainer import *
from SupContrastLoss import SupConLoss
from RankingLoss import MultipleNegativesSymmetricRankingLoss

/burg/dsi/users/ma4608/ajay-ta2-system/training_source/ajay-pausi-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-03 10:52:35.174716: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-03 10:52:58.894876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 10:53:01.928140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has alre

In [4]:
import torch

t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

In [8]:
t

47760867328

In [9]:
def train_datadreamer_ta2(fold, output_folder, used_loss):
    with DataDreamer(output_folder):
        dataset = DataSource(
            "Train Data",
            data=train_data_generator,
            total_num_rows=train_num_rows,
        )
        dev_dataset = DataSource(
            "Dev Data",
            data=dev_data_generator,
            total_num_rows=dev_num_rows,
        )
    
        trainer = get_luar_trainer()(
            "LUAR Trainer",
            model_name="./rrivera1849/",
            peft_config=LoraConfig(),
            trust_remote_code=True,
            device='cuda:0',
            dtype="bfloat16",
            force=True,
        )

        loss = SupConLoss if used_loss=='SupConLoss' else losses.MultipleNegativesSymmetricRankingLoss

        if used_loss == 'SupConLoss':
            trainer.train_with_labeled_pairs(
                train_anchors=dataset.output["anchors"],
                train_others= dataset.output["others"],
                train_labels=dataset.output["labels"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_others=dev_dataset.output["others"],
                validation_labels=dev_dataset.output["labels"],
                epochs=25,    
                batch_size=128,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )
        else:
            trainer.train_with_positive_pairs(
                train_anchors=dataset.output["anchors"],
                train_positives=dataset.output["positives"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_positives=dev_dataset.output["positives"],
                epochs=25,    
                batch_size=100,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )

### Train original TA2 system

In [4]:
fold = "data/train-test-dev split/{split}/Official Query-candidate format/"
used_loss= "supervised_contrastive_loss"

In [5]:
train_num_rows, train_data_generator = get_data_generator(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

Dataset Statistics: cross_genre_all train None 575 4142
Dataset Statistics: cross_genre_all dev None 79 635


In [6]:
train_datadreamer_ta2(fold, './original_ta2_model', 'MultipleNegativesSymmetricRankingLoss')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./original_ta2_model
[ 🤖 DataDreamer 💤 ] Step 'Train Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Train Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Positives' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Positives' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' will run lazi

The repository for ./rrivera1849/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./rrivera1849/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Finished loading.
[ 🤖 DataDreamer 💤 ] [ 🤗 Accelerate] Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Positives' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Eval Epoch: 0.0 -- {'loss': 3.1012659072875977, 'runtime': 6.0931, 'samples_per_second': 25.931, 'steps_per_second': 0.328}
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./original_ta2_model


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 44.48 GiB of which 339.25 MiB is free. Including non-PyTorch memory, this process has 44.15 GiB memory in use. Of the allocated memory 43.61 GiB is allocated by PyTorch, and 353.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Train original TA2 system with Supervised Contrastive Loss:

In [9]:
train_num_rows, train_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

Dataset Statistics: cross_genre_all train None 575 4142
Dataset Statistics: cross_genre_all dev None 79 635


In [10]:
train_datadreamer_ta2(fold, './supcon_ta2_model', 'SupConLoss')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./supcon_ta2_model
[ 🤖 DataDreamer 💤 ] Step 'Train Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Train Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Others' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Others' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' will run lazily. 🥱
[ 

The repository for ./rrivera1849/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./rrivera1849/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Finished loading.
[ 🤖 DataDreamer 💤 ] [ 🤗 Accelerate] Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Others' finished running lazily. 🎉


[(644.0, 4), (903.0, 2), (266.0, 4), (523.0, 3), (1294.0, 2), (529.0, 2), (1301.0, 2), (277.0, 2), (1305.0, 3), (796.0, 3), (416.0, 2), (931.0, 4), (165.0, 3), (39.0, 2), (680.0, 3), (1064.0, 2), (681.0, 2), (812.0, 3), (173.0, 5), (562.0, 2), (312.0, 2), (444.0, 3), (1345.0, 2), (194.0, 6), (580.0, 3), (211.0, 2), (215.0, 6), (88.0, 4), (87.0, 2), (882.0, 2), (117.0, 2), (251.0, 4), (511.0, 4)]
[(902.0, 2), (1031.0, 2), (1020.0, 2), (148.0, 3), (1301.0, 4), (278.0, 3), (792.0, 2), (1305.0, 3), (152.0, 2), (157.0, 4), (1309.0, 5), (413.0, 2), (551.0, 4), (681.0, 3), (680.0, 2), (1076.0, 2), (1334.0, 3), (312.0, 4), (444.0, 6), (194.0, 2), (580.0, 3), (1350.0, 2), (202.0, 2), (78.0, 3), (848.0, 2), (81.0, 3), (82.0, 4), (211.0, 3), (215.0, 5), (97.0, 2), (484.0, 3), (1261.0, 2), (1264.0, 2), (116.0, 2), (251.0, 2), (764.0, 2)]


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Eval Epoch: 0.0 -- {'loss': 191.56552124023438, 'runtime': 4.5626, 'samples_per_second': 34.629, 'steps_per_second': 0.438}


...

[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Eval Epoch: 7.444444444444445 -- {'loss': 11.764237403869629, 'runtime': 4.6172, 'samples_per_second': 34.22, 'steps_per_second': 0.433}
[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Train Epoch: 7.444444444444445 -- {'runtime': 399.3811, 'samples_per_second': 71.986, 'steps_per_second': 0.563, 'loss': 6.666211252862757}


[(645.0, 2), (902.0, 3), (1031.0, 2), (148.0, 2), (277.0, 2), (152.0, 6), (1178.0, 2), (796.0, 3), (1182.0, 2), (416.0, 2), (1192.0, 2), (680.0, 2), (551.0, 2), (173.0, 2), (819.0, 3), (1334.0, 2), (311.0, 2), (312.0, 6), (184.0, 2), (444.0, 5), (194.0, 2), (580.0, 2), (1229.0, 2), (84.0, 2), (212.0, 2), (215.0, 4), (738.0, 2), (484.0, 2), (1261.0, 4), (1264.0, 2), (241.0, 2), (116.0, 2), (1142.0, 2), (251.0, 4), (764.0, 2)]


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Loading trained model from disk...


The repository for ./rrivera1849/ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./rrivera1849/.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (./rrivera1849/)] Finished loading.
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./supcon_ta2_model
